In [28]:
dataurlspath = 'https://bulkdata.uspto.gov/data/patent/application/redbook/bibliographic/2014/'
dataurls =[
'ipab20140102_wk01.zip',
'ipab20140109_wk02.zip',
'ipab20140116_wk03.zip',
'ipab20140123_wk04.zip',
'ipab20140130_wk05.zip',
'ipab20140206_wk06.zip',
'ipab20140213_wk07.zip',
'ipab20140220_wk08.zip',
'ipab20140227_wk09.zip',
'ipab20140306_wk10.zip',
'ipab20140313_wk11.zip',
'ipab20140320_wk12.zip',
'ipab20140327_wk13.zip',
'ipab20140403_wk14.zip',
'ipab20140410_wk15.zip',
'ipab20140417_wk16.zip',
'ipab20140424_wk17.zip',
'ipab20140501_wk18.zip',
'ipab20140508_wk19.zip',
'ipab20140515_wk20.zip',
'ipab20140522_wk21.zip',
'ipab20140529_wk22.zip',
'ipab20140605_wk23.zip',
'ipab20140612_wk24.zip',
'ipab20140619_wk25.zip',
'ipab20140626_wk26.zip',
'ipab20140703_wk27.zip',
'ipab20140710_wk28.zip',
'ipab20140717_wk29.zip',
'ipab20140724_wk30.zip',
'ipab20140731_wk31.zip',
'ipab20140807_wk32.zip',
'ipab20140814_wk33.zip',
'ipab20140821_wk34.zip',
'ipab20140828_wk35.zip',
'ipab20140904_wk36.zip',
'ipab20140911_wk37.zip',
'ipab20140918_wk38.zip',
'ipab20140925_wk39.zip',
'ipab20141002_wk40.zip',
'ipab20141009_wk41.zip',
'ipab20141016_wk42.zip',
'ipab20141023_wk43.zip',
'ipab20141030_wk44.zip',
'ipab20141106_wk45.zip',
'ipab20141113_wk46.zip',
'ipab20141120_wk47.zip',
'ipab20141127_wk48.zip',
'ipab20141204_wk49.zip',
'ipab20141211_wk50.zip',
'ipab20141218_wk51.zip',
'ipab20141225_wk52.zip',
]

In [29]:
def xmltocsv(inputfilename, filename):
    
    fName = '../data/output/patent_data_' + filename[:-4] + '.csv'
    fw = open(fName, 'w')
    w = csv.writer(fw, delimiter = ',')

    dfilepath = inputfilename
    dfilepathtmp = dfilepath + 'tmp'
    !grep -v '<?xml version="1.0" encoding="UTF-8"?>' $dfilepath > $dfilepathtmp; mv $dfilepathtmp $dfilepath
    !sed -e '/^<!DOCTYPE/d' $dfilepath > $dfilepathtmp; mv $dfilepathtmp $dfilepath
    !sed -i '1 i\<items>' $dfilepath
    !sed -i -e '$a</items>' $dfilepath
    
    pub_doc_number = ''
    appl_doc_number = ''
    appl_country = ''
    invention_title = ''
    abstract = ''
    for event, element in etree.iterparse(inputfilename, tag="us-patent-application"):
        for child in element:
            
            if child.tag == 'us-bibliographic-data-application':
                for child1 in child:
                    if child1.tag == 'publication-reference':
                        if child1[0][1].tag == 'doc-number':
                            pub_doc_number = child1[0][1].text
                    
                    if child1.tag == 'application-reference':
                        if child1[0][1].tag == 'doc-number':
                            appl_country = child1[0][0].text 
                            appl_doc_number = child1[0][1].text       
                            
                    if child1.tag == 'invention-title':
                        for x in child1.itertext():
                            if x and not x.strip().isdigit(): # to remove the break numbers like 102 etc
                                invention_title = invention_title + x

            if child.tag == 'abstract':
                for x in child[0].itertext():
                    if x and not x.strip().isdigit(): # to remove the break numbers like 102 etc
                        abstract = abstract + x
                #print(abstract)

            if child.tag == 'abstract':
                w.writerow([appl_country, pub_doc_number, appl_doc_number, invention_title, abstract.strip()])
                pub_doc_number = ''
                appl_doc_number = ''
                appl_country = ''
                invention_title = ''
                abstract = ''
                break
 
        element.clear()
    fw.close() 

In [30]:
import csv
import time
import os
import zipfile
import lxml.etree as etree, lxml.html
import string


indir = '../data'
loaddir = '../data/loadzone'
loaddelete = '../data/loadzone/*'

#for i in range(0,1):    
for i in range(0,len(dataurls)):
    print('Downloading file:' + str(i))
    #down load the file and 
    dataurl = dataurlspath + dataurls[i]
    !curl -L $dataurl -o '../data/datafile.zip'
    
    for path, subdirs, files in os.walk(indir):
        
        for f in files:
            filepath = os.path.join(path, f)
            #print(filepath)
            !rm -r $loaddelete
            zip_ref = zipfile.ZipFile(filepath, 'r')
            zip_ref.extractall('../data/loadzone')
            zip_ref.close()

            datadir = '../data/loadzone'
            for dpath, dsubdirs, dfiles in os.walk(datadir):
                for df in dfiles:
                    if df.endswith('xml'):
                        dfilepath = os.path.join(dpath, df)
                        print('Starting file: ',dfilepath )
                        xmltocsv(dfilepath,dataurls[i])
                        print('Loaded file: ', dfilepath )
        break              

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3452k  100 3452k    0     0  7320k      0 --:--:-- --:--:-- --:--:-- 7330k
Starting file:  ../data/loadzone/ipab20140102.xml
Loaded file:  ../data/loadzone/ipab20140102.xml
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2993k  100 2993k    0     0  8547k      0 --:--:-- --:--:-- --:--:-- 8528k
Starting file:  ../data/loadzone/ipab20140109.xml
Loaded file:  ../data/loadzone/ipab20140109.xml
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3227k  100 3227k    0     0  8005k      0 --:--:-- --:--:-- --:--:-- 8008k
Starting file:  ../data/loadzone/ipab20140116.xml
Loaded file:  ../data/loadzone/ipab20140116

100 3781k  100 3781k    0     0  8201k      0 --:--:-- --:--:-- --:--:-- 8219k
Starting file:  ../data/loadzone/ipab20140612.xml
Loaded file:  ../data/loadzone/ipab20140612.xml
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4072k  100 4072k    0     0  1934k      0  0:00:02  0:00:02 --:--:-- 1934k
Starting file:  ../data/loadzone/ipab20140619.xml
Loaded file:  ../data/loadzone/ipab20140619.xml
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3948k  100 3948k    0     0  7953k      0 --:--:-- --:--:-- --:--:-- 7943k
Starting file:  ../data/loadzone/ipab20140626.xml
Loaded file:  ../data/loadzone/ipab20140626.xml
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  S

100 3542k  100 3542k    0     0  2570k      0  0:00:01  0:00:01 --:--:-- 2571k
Starting file:  ../data/loadzone/ipab20141120.xml
Loaded file:  ../data/loadzone/ipab20141120.xml
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3619k  100 3619k    0     0  7804k      0 --:--:-- --:--:-- --:--:-- 7816k
Starting file:  ../data/loadzone/ipab20141127.xml
Loaded file:  ../data/loadzone/ipab20141127.xml
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3913k  100 3913k    0     0  8392k      0 --:--:-- --:--:-- --:--:-- 8381k
Starting file:  ../data/loadzone/ipab20141204.xml
Loaded file:  ../data/loadzone/ipab20141204.xml
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  S

In [27]:
# dataurlspath = 'https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/2014/'
# dataurls =[
# 'ipa140102.zip',
# 'ipa140109.zip',
# 'ipa140116.zip',
# 'ipa140123.zip',
# 'ipa140130.zip',
# 'ipa140206.zip',
# 'ipa140213.zip',
# 'ipa140220.zip',
# 'ipa140227.zip',
# 'ipa140306.zip',
# 'ipa140313.zip',
# 'ipa140320.zip',
# 'ipa140327.zip',
# 'ipa140403.zip',
# 'ipa140410.zip',
# 'ipa140417.zip',
# 'ipa140424.zip',
# 'ipa140501.zip',
# 'ipa140508.zip',
# 'ipa140515.zip',
# 'ipa140522.zip',
# 'ipa140529.zip',
# 'ipa140605.zip',
# 'ipa140612.zip',
# 'ipa140619.zip',
# 'ipa140626.zip',
# 'ipa140703.zip',
# 'ipa140710.zip',
# 'ipa140717.zip',
# 'ipa140724.zip',
# 'ipa140731.zip',
# 'ipa140807.zip',
# 'ipa140814.zip',
# 'ipa140821.zip',
# 'ipa140828.zip',
# 'ipa140904.zip',
# 'ipa140911.zip',
# 'ipa140918.zip',
# 'ipa140925.zip',
# 'ipa141002.zip',
# 'ipa141009.zip',
# 'ipa141016.zip',
# 'ipa141023.zip',
# 'ipa141030.zip',
# 'ipa141106.zip',
# 'ipa141113.zip',
# 'ipa141120.zip',
# 'ipa141127.zip',
# 'ipa141204.zip',
# 'ipa141211.zip',
# 'ipa141218.zip',
# 'ipa141225.zip'
# ]